In [61]:
import pandas as pd
df=pd.read_csv('/Users/joanna/Desktop/作品集/deleted_outlier-2.csv')
df.columns

Index(['State', 'Sex', 'GeneralHealth', 'PhysicalHealthDays',
       'MentalHealthDays', 'LastCheckupTime', 'PhysicalActivities',
       'SleepHours', 'RemovedTeeth', 'HadHeartAttack', 'HadAngina',
       'HadStroke', 'HadAsthma', 'HadSkinCancer', 'HadCOPD',
       'HadDepressiveDisorder', 'HadKidneyDisease', 'HadArthritis',
       'HadDiabetes', 'DeafOrHardOfHearing', 'BlindOrVisionDifficulty',
       'DifficultyConcentrating', 'DifficultyWalking',
       'DifficultyDressingBathing', 'DifficultyErrands', 'SmokerStatus',
       'ECigaretteUsage', 'ChestScan', 'RaceEthnicityCategory', 'AgeCategory',
       'HeightInMeters', 'WeightInKilograms', 'BMI', 'AlcoholDrinkers',
       'HIVTesting', 'FluVaxLast12', 'PneumoVaxEver', 'TetanusLast10Tdap',
       'HighRiskLastYear', 'CovidPos', 'cal_BMI'],
      dtype='object')

In [62]:
X=df.drop('HadHeartAttack',axis=1)
y=df['HadHeartAttack'].replace({'No': 0, 'Yes': 1})
y

0         0
1         0
2         0
3         0
4         0
         ..
238480    0
238481    0
238482    0
238483    0
238484    1
Name: HadHeartAttack, Length: 238485, dtype: int64

In [63]:
X['AgeCategory']=X['AgeCategory'].replace({
    'Age 18 to 24': 0,
    'Age 25 to 29': 1,
    'Age 30 to 34': 2,
    'Age 35 to 39': 3,
    'Age 40 to 44': 4,
    'Age 45 to 49': 5,
    'Age 50 to 54': 6,
    'Age 55 to 59': 7,
    'Age 60 to 64': 8,
    'Age 65 to 69': 9,
    'Age 70 to 74': 10,
    'Age 75 to 79': 11,
    'Age 80 or older': 12})
X['GeneralHealth']=X['GeneralHealth'].replace({'Excellent':4,'Very good':3,'Good':2,'Fair':1,'Poor':0 })

In [64]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
import pandas as pd
from sklearn.preprocessing import StandardScaler

# 要做 one-hot encoding 的欄位
onehot_cols = ['State', 'Sex','LastCheckupTime', 'PhysicalActivities','RemovedTeeth','HadAngina',
       'HadStroke', 'HadAsthma', 'HadSkinCancer', 'HadCOPD',
       'HadDepressiveDisorder', 'HadKidneyDisease', 'HadArthritis',
       'HadDiabetes', 'DeafOrHardOfHearing', 'BlindOrVisionDifficulty',
       'DifficultyConcentrating', 'DifficultyWalking',
       'DifficultyDressingBathing', 'DifficultyErrands', 'SmokerStatus',
       'ECigaretteUsage', 'ChestScan', 'RaceEthnicityCategory','AlcoholDrinkers',
       'HIVTesting', 'FluVaxLast12', 'PneumoVaxEver', 'TetanusLast10Tdap',
       'HighRiskLastYear', 'CovidPos']

# 要做標準化的欄位
scale_cols = ['PhysicalHealthDays','MentalHealthDays','SleepHours',
              'HeightInMeters', 'WeightInKilograms', 'BMI','cal_BMI']

# Step 1: One-hot encoding
X_dummies = pd.get_dummies(X, columns=onehot_cols, drop_first=True)

# Step 2: Standardization
scaler = StandardScaler()
X_dummies[scale_cols] = scaler.fit_transform(X_dummies[scale_cols])

# 最終轉換後的 X
X_new = X_dummies


In [52]:
# Decision tree

In [65]:
from sklearn.tree import DecisionTreeClassifier
tree=DecisionTreeClassifier(max_depth=3,class_weight='balanced',random_state=0)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_new,y,test_size=0.3,random_state=0)
tree.fit(X_train,y_train)
y_pred=tree.predict(X_test)
from sklearn.metrics import classification_report
report=classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.99      0.66      0.79     67668
           1       0.13      0.85      0.22      3878

    accuracy                           0.67     71546
   macro avg       0.56      0.76      0.51     71546
weighted avg       0.94      0.67      0.76     71546



In [66]:
from sklearn.model_selection import cross_val_score

acc_tree=cross_val_score(tree,X_new,y,cv=10,scoring='accuracy')
print(acc_tree.mean())
re_tree=cross_val_score(tree,X_new,y,cv=10,scoring='recall')
print(re_tree.mean())
pre_tree=cross_val_score(tree,X_new,y,cv=10,scoring='precision')
print(pre_tree.mean())
f1_tree=cross_val_score(tree,X_new,y,cv=10,scoring='f1')
print(f1_tree.mean())
auc_tree=cross_val_score(tree,X_new,y,cv=10,scoring='roc_auc_ovr')
print(auc_tree.mean())

0.6947817096576921
0.8326640926640925
0.13269581641359135
0.22885261016759037
0.8463662234022227


In [67]:
# test if overfit:訓練集表現明顯比test高很多
from sklearn.metrics import accuracy_score, f1_score
y_train_pred = tree.predict(X_train)
y_test_pred = tree.predict(X_test)
print("Train Accuracy:", accuracy_score(y_train, y_train_pred))
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
print("Train F1:", f1_score(y_train, y_train_pred))
print("Test F1:", f1_score(y_test, y_test_pred))


Train Accuracy: 0.6743241543318218
Test Accuracy: 0.6748385653984849
Train F1: 0.21983698771668006
Test F1: 0.22042758528248776


In [68]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV
from sklearn.metrics import classification_report

# 拆分資料
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.3, random_state=0)

# 建立基本模型
tree = DecisionTreeClassifier(class_weight='balanced',random_state=0)

# 設定要搜尋的參數組合
param_grid = {
    'max_depth': [10, 15, 20, 25, 30],             # 防止過擬合，控制樹高度
    'min_samples_split': [10, 50, 100],            # 每個節點至少幾筆樣本才可再切
    'min_samples_leaf': [5, 10, 50]                # 每個葉節點至少樣本數，提升穩定性
}

# 建立 GridSearchCV 模型，使用 F1 (weighted) 當作評估標準
grid = RandomizedSearchCV(tree, param_grid, cv=5, scoring='f1', n_jobs=-1)
grid.fit(X_train, y_train)

# 預測與報告
y_pred = grid.best_estimator_.predict(X_test)
print("最佳參數：", grid.best_params_)
print("分類報告：\n", classification_report(y_test, y_pred))


最佳參數： {'min_samples_split': 10, 'min_samples_leaf': 10, 'max_depth': 10}
分類報告：
               precision    recall  f1-score   support

           0       0.98      0.82      0.89     67668
           1       0.19      0.75      0.31      3878

    accuracy                           0.82     71546
   macro avg       0.59      0.79      0.60     71546
weighted avg       0.94      0.82      0.86     71546



In [69]:
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
rus=RandomUnderSampler(replacement=False)#sampling will be done without replacement
X_train_subsample,y_train_subsample=rus.fit_resample(X_train,y_train)
print(X_train.shape)
print(y_train.shape)
print(X_train_subsample.shape)
#print class distribution of resampled data ＃np.bincount 函數計算了每個類別出現的次數
print(np.bincount(y_train_subsample))

(166939, 108)
(166939,)
(18144, 108)
[9072 9072]


In [70]:
from imblearn.pipeline import make_pipeline as make_imb_pipeline
from sklearn.model_selection import cross_validate
undersample_pipe=make_imb_pipeline(RandomUnderSampler(),tree)
scores=cross_validate(undersample_pipe,X_train,y_train,cv=10,scoring=('accuracy','recall','precision','f1','roc_auc'))
pd.DataFrame(scores)[['test_accuracy','test_recall','test_precision','test_f1','test_roc_auc']].mean()

test_accuracy     0.731081
test_recall       0.731483
test_precision    0.135187
test_f1           0.228193
test_roc_auc      0.731271
dtype: float64

In [71]:
from imblearn.over_sampling import RandomOverSampler
ros=RandomOverSampler()
X_train_oversample,y_train_oversample=ros.fit_resample(X_train,y_train)
print(X_train.shape)
print(y_train.shape)
print(X_train_oversample.shape)
#print class distribution of resampled data
print(np.bincount(y_train_oversample))

(166939, 108)
(166939,)
(315734, 108)
[157867 157867]


In [72]:
oversample_pipe=make_imb_pipeline(RandomOverSampler(),tree)
scores=cross_validate(oversample_pipe,X_train,y_train,cv=5,scoring=('accuracy','recall','precision','f1','roc_auc'))
pd.DataFrame(scores)[['test_accuracy','test_recall','test_precision','test_f1','test_roc_auc']].mean()

KeyboardInterrupt: 

In [ ]:
from imblearn.over_sampling import SMOTE
smote=SMOTE()
X_train_smote,y_train_smote=smote.fit_resample(X_train,y_train)
print(X_train.shape)
print(y_train.shape)
print(X_train_smote.shape)
#print class distribution of resampled data
print(np.bincount(y_train_smote))

In [ ]:
smote_pipe=make_imb_pipeline(SMOTE(),tree)
scores=cross_validate(smote_pipe,X_train,y_train,cv=5,scoring=('accuracy','recall','precision','f1','roc_auc'))
pd.DataFrame(scores)[['test_accuracy','test_recall','test_precision','test_f1','test_roc_auc']].mean()

In [99]:
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import make_pipeline as make_imb_pipeline
from imblearn.over_sampling import SMOTE
import pandas as pd

# 定義模型與參數
tree = DecisionTreeClassifier(class_weight='balanced',random_state=0)
param_grid = {
    'decisiontreeclassifier__max_depth': [5, 10, 15,20,25,30],
    'decisiontreeclassifier__min_samples_split': [5, 10,50,100],
    'decisiontreeclassifier__min_samples_leaf': [5,10,50]
}

# 建立 pipeline
smote_pipe1 = make_imb_pipeline(SMOTE(random_state=42), tree)

# 使用 GridSearchCV 包住 pipeline
grid1 =RandomizedSearchCV(
    smote_pipe,
    param_distributions=param_grid,
    cv=5,
    scoring='f1',  # 主評估指標，可改為 'roc_auc'、'accuracy' 等
    n_jobs=-1,
    verbose=1,
    
)

# 執行 grid search
grid1.fit(X_train, y_train)


Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('smote', SMOTE(random_state=42)),
                                             ('decisiontreeclassifier',
                                              DecisionTreeClassifier(class_weight='balanced',
                                                                     random_state=0))]),
                   n_jobs=-1,
                   param_distributions={'decisiontreeclassifier__max_depth': [5,
                                                                              10,
                                                                              15,
                                                                              20,
                                                                              25,
                                                                              30],
                                        'decisiontreeclassifier__min_samples_leaf': [5,
                                                                                     10,
                                                                                     50],
                                        'decisiontreeclassifier__min_samples_split': [5,
                                                                                      10,
                                                                                      50,
                                                                                      100]},
                   scoring='f1', verbose=1)

In [100]:
#******
#smote grid result
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

# 測試集預測（使用最佳模型）
y_pred = grid1.best_estimator_.predict(X_test)

# 輸出完整分類報告（每個類別的 precision / recall / f1）
print("最佳參數：", grid.best_params_)
print("測試集分類報告：\n", classification_report(y_test, y_pred, digits=4))

print("Accuracy:", round(accuracy_score(y_test, y_pred), 4))
print("Precision:", round(precision_score(y_test, y_pred), 4))
print(" Recall :", round(recall_score(y_test, y_pred), 4))
print("F1-score:", round(f1_score(y_test, y_pred), 4))

#策略 1：再微調 Decision Tree 的 min_samples_leaf 往上調（抑制過擬合）
#策略 2：改用 Random Forest / XGBoost → 更穩定、泛化能力好
#策略 3：改變樣本比例（減少 SMOTE 合成量）或嘗試 SMOTEENN 平衡手法


最佳參數： {'min_samples_split': 10, 'min_samples_leaf': 10, 'max_depth': 10}
測試集分類報告：
               precision    recall  f1-score   support

           0     0.9676    0.8794    0.9214      6766
           1     0.1889    0.4884    0.2724       389

    accuracy                         0.8581      7155
   macro avg     0.5783    0.6839    0.5969      7155
weighted avg     0.9253    0.8581    0.8861      7155

Accuracy: 0.8581
Precision: 0.1889
 Recall : 0.4884
F1-score: 0.2724


In [101]:
# 7. 特徵重要性
# 從 pipeline 中抓出最後的 classifier 模型
best_model1 = grid1.best_estimator_.named_steps['decisiontreeclassifier']
importances1 = best_model1.feature_importances_

# 如果你有 feature name
feature_names = X_train.columns if hasattr(X_train, 'columns') else [f"Feature {i}" for i in range(X_train.shape[1])]
feature_importance1_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances1
}).sort_values(by='Importance', ascending=False)

print("\n📊 特徵重要性（Feature Importances）:")
print(feature_importance1_df.head(20))


📊 特徵重要性（Feature Importances）:
                                               Feature  Importance
69                           RemovedTeeth_None of them    0.245330
70                                       HadAngina_Yes    0.158299
0                                        GeneralHealth    0.080400
5                                       HeightInMeters    0.080354
4                                          AgeCategory    0.055679
3                                           SleepHours    0.041915
102               TetanusLast10Tdap_Yes, received Tdap    0.030887
90              ECigaretteUsage_Not at all (right now)    0.025725
1                                   PhysicalHealthDays    0.021733
103  TetanusLast10Tdap_Yes, received tetanus shot b...    0.018534
89                           SmokerStatus_Never smoked    0.018371
107                                       CovidPos_Yes    0.017565
104  TetanusLast10Tdap_Yes, received tetanus shot, ...    0.016321
6                              

In [104]:
# 假設你決定用前 20 個特徵
top_features1 = feature_importance1_df.head(20)['Feature'].tolist()
X_selected= X_new[top_features1]

# 重新切分
X_train_sel, X_test_sel, y_train_sel, y_test_sel = train_test_split(X_selected, y, test_size=0.3, stratify=y, random_state=42)
smote_pipe1.fit(X_train_sel, y_train_sel)

# 評估新模型表現
y_pred_sel = smote_pipe1.predict(X_test_sel)
print(classification_report(y_test_sel, y_pred_sel))

              precision    recall  f1-score   support

           0       0.96      0.92      0.94     67661
           1       0.22      0.36      0.27      3885

    accuracy                           0.89     71546
   macro avg       0.59      0.64      0.61     71546
weighted avg       0.92      0.89      0.91     71546



In [105]:
top_features1

['RemovedTeeth_None of them',
 'HadAngina_Yes',
 'GeneralHealth',
 'HeightInMeters',
 'AgeCategory',
 'SleepHours',
 'TetanusLast10Tdap_Yes, received Tdap',
 'ECigaretteUsage_Not at all (right now)',
 'PhysicalHealthDays',
 'TetanusLast10Tdap_Yes, received tetanus shot but not sure what type',
 'SmokerStatus_Never smoked',
 'CovidPos_Yes',
 'TetanusLast10Tdap_Yes, received tetanus shot, but not Tdap',
 'WeightInKilograms',
 'RaceEthnicityCategory_Hispanic',
 'RemovedTeeth_6 or more, but not all',
 'HadSkinCancer_Yes',
 'BMI',
 'AlcoholDrinkers_Yes',
 'PhysicalActivities_Yes']

In [110]:
#*****
#decision tree random over cv=5 RandomizedSearchCV
from imblearn.pipeline import make_pipeline as make_imb_pipeline
from imblearn.over_sampling import RandomOverSampler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score,roc_auc_score


# 定義模型與參數
tree = DecisionTreeClassifier(class_weight='balanced', random_state=0)
param_grid = {
    'decisiontreeclassifier__max_depth': [5, 10, 15, 20, 25, 30],
    'decisiontreeclassifier__min_samples_split': [5, 10, 50, 100],
    'decisiontreeclassifier__min_samples_leaf': [5, 10, 50]
}

# 建立 pipeline：改用 RandomOverSampler
oversample_pipe = make_imb_pipeline(RandomOverSampler(random_state=42), tree)

# 使用 GridSearchCV 包住 pipeline
grid2 = RandomizedSearchCV(
    oversample_pipe,
    param_distributions=param_grid,
    cv=5,
    scoring='f1',  # 可根據問題選擇 'f1', 'roc_auc', 'accuracy' 等
    n_jobs=-1,
    verbose=1
)

# 執行 grid search
grid2.fit(X_train, y_train)

# 測試集預測
y_pred = grid.best_estimator_.predict(X_test)

# 輸出報告
print("最佳參數：", grid.best_params_)
print("測試集分類報告：\n", classification_report(y_test, y_pred, digits=4))
print("Accuracy:", round(accuracy_score(y_test, y_pred), 4))
print("Precision:", round(precision_score(y_test, y_pred), 4))
print(" Recall :", round(recall_score(y_test, y_pred), 4))
print("F1-score:", round(f1_score(y_test, y_pred), 4))
print("ROC AUC:",round(roc_auc_score(y_test, y_pred), 4))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
最佳參數： {'min_samples_split': 10, 'min_samples_leaf': 10, 'max_depth': 10}
測試集分類報告：
               precision    recall  f1-score   support

           0     0.9838    0.8243    0.8970      6766
           1     0.1999    0.7635    0.3168       389

    accuracy                         0.8210      7155
   macro avg     0.5918    0.7939    0.6069      7155
weighted avg     0.9412    0.8210    0.8654      7155

Accuracy: 0.821
Precision: 0.1999
 Recall : 0.7635
F1-score: 0.3168
ROC AUC: 0.7939


In [83]:
# 7. 特徵重要性
# 從 pipeline 中抓出最後的 classifier 模型
best_model2 = grid2.best_estimator_.named_steps['decisiontreeclassifier']
importances2 = best_model2.feature_importances_

# 如果你有 feature name
feature_names = X_train.columns if hasattr(X_train, 'columns') else [f"Feature {i}" for i in range(X_train.shape[1])]
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances2
}).sort_values(by='Importance', ascending=False)

print("\n📊 特徵重要性（Feature Importances）:")
print(feature_importance_df.head(20))


📊 特徵重要性（Feature Importances）:
                      Feature  Importance
70              HadAngina_Yes    0.568418
93              ChestScan_Yes    0.127730
4                 AgeCategory    0.095132
0               GeneralHealth    0.060272
71              HadStroke_Yes    0.029329
62                   Sex_Male    0.017523
6           WeightInKilograms    0.009980
89  SmokerStatus_Never smoked    0.008078
84      DifficultyWalking_Yes    0.006610
3                  SleepHours    0.006591
74                HadCOPD_Yes    0.005767
79            HadDiabetes_Yes    0.005684
5              HeightInMeters    0.005023
8                     cal_BMI    0.004636
2            MentalHealthDays    0.004339
98        AlcoholDrinkers_Yes    0.003689
7                         BMI    0.003621
1          PhysicalHealthDays    0.003534
68           RemovedTeeth_All    0.003344
81    DeafOrHardOfHearing_Yes    0.003334
